# Exhaustive search - Flowshop problem



### Table of content
- [Heuristics](#Heuristics)
- [Branch & Bound](#Branch-&-Bound)
- [Tests](#Tests)
   


### References
- [Finding an Optimal Sequence in the Flowshop Scheduling Using Johnson’s Algorithm](https://ijiset.com/vol2/v2s1/IJISET_V2_I1_50.pdf)
- [Benchmarks for Basic Scheduling Problems](http://mistic.heig-vd.ch/taillard/articles.dir/Taillard1993EJOR.pdf)

In [ ]:
import numpy as np
import matplotlib as plt
import itertools
import time
import math

# Heuristics

## NEH Algorithm

---



In [ ]:
def order_jobs_in_descending_order_of_total_completion_time(processing_times):
    total_completion_time = processing_times.sum(axis=1)
    return np.argsort(total_completion_time, axis=0).tolist()

In [ ]:
def insertion(sequence, position, value):
    new_seq = sequence[:]
    new_seq.insert(position, value)
    return new_seq

In [ ]:
def neh_algorithm(processing_times):
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)
    # Define the initial order
    J1, J2 = ordered_sequence[:2]
    sequence = [J1, J2] if evaluate_sequence([J1, J2], processing_times) < evaluate_sequence([J2, J1], processing_times) else [J2, J1]
    del ordered_sequence[:2]
    # Add remaining jobs
    for job in ordered_sequence:
        Cmax = float('inf')
        best_sequence = []
        for i in range(len(sequence)+1):
            new_sequence = insertion(sequence, i, job)
            Cmax_eval = evaluate_sequence(new_sequence, processing_times)
            if Cmax_eval < Cmax:
                Cmax = Cmax_eval
                best_sequence = new_sequence
        sequence = best_sequence
    return sequence, Cmax

# Branch & Bound

- Branch & Bound.
- Branch & Bound pure.

In [ ]:
# Define the Node structure of the seach tree that we will be using
class Node:
    def __init__(self, jobs, remaining_jobs, parent=None, lower_bound=1e100):
        self.jobs = jobs
        self.remaining_jobs = remaining_jobs
        self.parent = parent
        self.lower_bound = lower_bound
    def __str__(self):
        return f"Node(jobs={self.jobs}, remaining_jobs={self.remaining_jobs}, lower_bound={self.lower_bound})"
    def __lt__(self, other):
        # Define less than comparison for the priority queue
        return self.lower_bound < other.lower_bound

    def __eq__(self, other):
        # Define equality for the priority queue
        return self.lower_bound == other.lower_bound

In [ ]:
def evaluate_sequence(sequence, processing_times):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)

    # Check if the sequence is empty
    if num_jobs == 0:
        # Return a default value (you may choose 0 or another suitable value)
        return 0

    completion_times = np.zeros((num_jobs, num_machines))

    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]

    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]

    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]


In [ ]:
import numpy as np

def job_completion_times(jobs_matrix, sequence):
    num_jobs, num_machines = jobs_matrix.shape
    completion_matrix = np.zeros((len(sequence), num_machines))

    for i in range(len(sequence)):
        for j in range(num_machines):
            if i == 0 and j == 0:
                completion_matrix[0, j] = jobs_matrix[sequence[i], j]
            elif i == 0:
                completion_matrix[0, j] = jobs_matrix[sequence[i], j] + completion_matrix[0, j - 1]
            elif j == 0:
                completion_matrix[i, 0] = jobs_matrix[sequence[i], j] + completion_matrix[i - 1, 0]
            else:
                completion_matrix[i, j] = jobs_matrix[sequence[i], j] + max(completion_matrix[i - 1, j], completion_matrix[i, j - 1])

    return completion_matrix




In [ ]:

def fist_lower_baund(processing_times, column_sums, lign_sums):
    lb1 = column_sums[0] + min(lign_sums[1:])
    lb2 = column_sums[1] + min(lign_sums[:1] + lign_sums[2:])
    lb3 = column_sums[2] + min(lign_sums[:2] + lign_sums[3:])
    lb4 = column_sums[3] + min(lign_sums[:3] + lign_sums[4:])
    lb5 = column_sums[4] + min(lign_sums[:4] + lign_sums[5:])

    return max(lb1, lb2, lb3, lb4, lb5)


def first_LBA (processing_times,jobs,column_sums) :
    first_job=jobs[0]
    matrix =  processing_times[:, 1:]
    matrix = np.delete(matrix, jobs[0], axis=0)
    row_sums = np.sum(matrix, axis=1)
    lb1=column_sums[0] +np.min(row_sums)
    matrix =  matrix[:, 1:]
    row_sums = np.sum(matrix, axis=1)
    lb2=column_sums[1]+processing_times[first_job,0]+np.min(row_sums)
    matrix =  matrix[:, 1:]
    row_sums = np.sum(matrix, axis=1)
    lb3=processing_times[first_job,0]+processing_times[first_job,1]+column_sums[2]+np.min(row_sums)
    matrix =  matrix[:, 1:]
    row_sums = np.sum(matrix, axis=1)
    lb4=processing_times[first_job,0]+processing_times[first_job,1]+processing_times[first_job,2]+column_sums[3]+np.min(row_sums)

    lb5=processing_times[first_job,0]+processing_times[first_job,1]+processing_times[first_job,2]+processing_times[first_job,3]+column_sums[4]

    return max(lb1,lb2,lb3,lb4,lb5)

def LBA (processing_times,jobs,column_sums) :

    matrix =  processing_times[:, 1:]

    # Create a matrix without the jobs exist in the jobs list
    mask = np.ones(len(matrix), dtype=bool)
    mask[jobs] = False
    matrix = matrix[mask]
    #-------------------------------------------------------
    #
    row_sums = np.sum(matrix, axis=1)
    M =job_completion_times(processing_times, jobs)
    #---------------------lb1
    lb1=column_sums[0] +np.min(row_sums)


    #---------------------lb2

    lb2=M[len(jobs)-1,1]+np.sum(matrix[:, 0])
    matrix =  matrix[:, 1:]
    row_sums = np.sum(matrix, axis=1)
    lb2=lb2+np.min(row_sums)
    #---------------------lb3

    lb3=M[len(jobs)-1,2]+np.sum(matrix[:, 0])
    matrix =  matrix[:, 1:]
    row_sums = np.sum(matrix, axis=1)
    lb3=lb3+np.min(row_sums)
    #---------------------lb4
    lb4=M[len(jobs)-1,3]+np.sum(matrix[:, 0])
    matrix =  matrix[:, 1:]
    row_sums = np.sum(matrix, axis=1)
    lb4=lb4+np.min(row_sums)
    #---------------------lb5
    lb5=M[len(jobs)-1,4]+np.sum(matrix[:, 0])



    return max(lb1,lb2,lb3,lb4,lb5)




## Branch & Bound

In [ ]:
from typing import List

def branch_and_bound(processing_times, initial_solution, initial_cost):
    column_sums = [sum(col) for col in zip(*processing_times)]
    jobs, machines = processing_times.shape
    root_node = Node([], set(range(jobs)))
    best_solution = initial_solution.copy()
    best_solution_cost = initial_cost

    def dfs(node):

        nonlocal best_solution, best_solution_cost
        if ((len(node.jobs)!=0 and node.lower_bound < best_solution_cost) or (len(node.jobs)==0)):
            # Calculate the lower bounds for all remaining jobs
            lower_bounds = []
            for job in node.remaining_jobs:
                child_jobs = node.jobs + [job]
                child_remaining_jobs = node.remaining_jobs - {job}

                # Calculate the lower bound for the child node
                ev = evaluate_sequence(child_jobs, processing_times)
                if not child_remaining_jobs:
                    child_lower_bound = ev
                else:
                    child_lower_bound = LBA(processing_times, child_jobs, column_sums)

                if not child_remaining_jobs and job == max(node.remaining_jobs):
                    if child_lower_bound < best_solution_cost:
                        best_solution = child_jobs
                        best_solution_cost = child_lower_bound
                        print('new best solution', best_solution)
                        print('new best cost', best_solution_cost)
                    continue

                # If the child node is not a leaf, continue the depth-first search
                if child_lower_bound < best_solution_cost:
                    child_node = Node(child_jobs, child_remaining_jobs, parent=node, lower_bound=child_lower_bound)
                    lower_bounds.append(child_node)

            # Sort the child nodes based on their lower bounds
            sorted_nodes = sorted(lower_bounds, key=lambda x: x.lower_bound)
            for sorted_node in sorted_nodes:
                dfs(sorted_node)

    # Start the depth-first search with the root node
    dfs_stack = [root_node]
    i = 0
    while dfs_stack:
        current_node = dfs_stack.pop()
        dfs(current_node)
        i += 1

    return best_solution, best_solution_cost, i


## Tests

# random test 10*5

In [ ]:
instance_common = np.array([
    [71, 79, 85, 82, 83],
    [84, 71, 66, 68, 81],
    [78, 81, 75, 72, 87],
    [78, 75, 66, 72, 88],
    [72, 88, 83, 85, 88],
    [86, 88, 79, 82, 78],
    [75, 66, 86, 78, 78],
    [80, 79, 66, 83, 78],
    [73, 73, 67, 77, 71],
    [80, 77, 83, 78, 67],
])

In [ ]:
instance_common.shape

(10, 5)

In [ ]:
initial_solution = [i for i in range(10)]
initial_cost = evaluate_sequence(initial_solution, instance_common)

start_time = time.time()
best_solution, best_cost,i= branch_and_bound(instance_common, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'Results of Branch & Bound:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')

print(f'Elapsed time of {elapsed_time} seconds.')

new best solution [1, 8, 3, 6, 2, 0, 4, 5, 7, 9]
new best cost 1115.0
new best solution [1, 8, 3, 6, 0, 2, 4, 5, 7, 9]
new best cost 1110.0
new best solution [1, 3, 6, 0, 2, 7, 8, 4, 5, 9]
new best cost 1109.0
new best solution [1, 3, 6, 0, 2, 7, 4, 8, 5, 9]
new best cost 1108.0
new best solution [8, 3, 6, 2, 7, 1, 0, 4, 5, 9]
new best cost 1107.0
new best solution [8, 3, 6, 2, 1, 0, 4, 5, 7, 9]
new best cost 1105.0
new best solution [3, 6, 2, 0, 1, 8, 4, 5, 7, 9]
new best cost 1104.0
new best solution [3, 2, 7, 6, 8, 0, 1, 4, 5, 9]
new best cost 1102.0
Results of Branch & Bound:
Best sequence is [3, 2, 7, 6, 8, 0, 1, 4, 5, 9] with a makespan of 1102.0.
Elapsed time of 0.49977922439575195 seconds.


# test instance 3

In [ ]:
import numpy as np
#-----------------------------------------------instance 3
data_matrix = [
    [77, 94, 9, 57, 29, 79, 55, 73, 65, 86, 25, 39, 76, 24, 38, 5, 91, 29, 22, 27],
    [39, 31, 46, 18, 93, 58, 85, 58, 97, 10, 79, 93, 2, 87, 17, 18, 10, 50, 8, 26],
    [14, 21, 15, 10, 85, 46, 42, 18, 36, 2, 44, 89, 6, 3, 1, 43, 81, 57, 76, 59],
    [11, 2, 36, 30, 89, 10, 88, 22, 31, 9, 43, 91, 26, 3, 75, 99, 63, 83, 70, 84],
    [83, 13, 84, 46, 20, 33, 74, 42, 33, 71, 32, 48, 42, 99, 7, 54, 8, 73, 30, 75]
]
matrix = np.array(data_matrix)

matrix = np.transpose(matrix)

neh_solution =neh_algorithm(matrix)

print(neh_solution[0])


[2, 19, 17, 4, 8, 0, 16, 13, 15, 7, 6, 18, 9, 11, 10, 5, 3, 14, 12, 1]


In [ ]:
initial_solution = neh_solution[0]
initial_cost = neh_solution[1]

start_time = time.time()
best_solution, best_cost, i = branch_and_bound(matrix, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'Results of Branch & Bound:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
#print(f'No. Nodes visited is {i}.')
print(f'Elapsed time of {elapsed_time} seconds.')

new best solution [2, 3, 9, 14, 18, 13, 12, 15, 17, 19, 6, 0, 4, 11, 5, 7, 16, 10, 8, 1]
new best cost 1156.0
new best solution [2, 3, 9, 14, 15, 10, 0, 18, 13, 12, 19, 17, 4, 6, 11, 7, 16, 5, 8, 1]
new best cost 1153.0
new best solution [2, 3, 9, 14, 15, 10, 0, 18, 13, 12, 19, 17, 6, 11, 4, 7, 16, 5, 8, 1]
new best cost 1151.0
new best solution [2, 3, 9, 14, 15, 10, 0, 18, 13, 12, 19, 17, 11, 6, 4, 7, 16, 5, 8, 1]
new best cost 1149.0
new best solution [2, 3, 9, 14, 15, 10, 0, 19, 6, 13, 12, 18, 17, 11, 4, 7, 16, 5, 8, 1]
new best cost 1147.0
new best solution [2, 3, 9, 14, 15, 10, 0, 19, 17, 6, 12, 13, 18, 11, 4, 7, 16, 5, 8, 1]
new best cost 1144.0
new best solution [2, 3, 9, 14, 15, 13, 12, 19, 17, 6, 0, 18, 4, 11, 5, 7, 16, 10, 8, 1]
new best cost 1143.0
new best solution [2, 3, 9, 15, 10, 0, 19, 6, 7, 18, 17, 4, 13, 12, 16, 11, 8, 5, 1, 14]
new best cost 1141.0
new best solution [2, 3, 9, 15, 10, 0, 19, 6, 7, 18, 17, 11, 12, 13, 16, 4, 8, 5, 1, 14]
new best cost 1133.0
new best s

# test instance 2

In [ ]:
import numpy as np
#-----------------------------------------------instance 2
matrix = np.array([
    [26, 38, 27, 88, 95, 55, 54, 63, 23, 45, 86, 43, 43, 40, 37, 54, 35, 59, 43, 50],
    [59, 62, 44, 10, 23, 64, 47, 68, 54, 9, 30, 31, 92, 7, 14, 95, 76, 82, 91, 37],
    [78, 90, 64, 49, 47, 20, 61, 93, 36, 47, 70, 54, 87, 13, 40, 34, 55, 13, 11, 5],
    [88, 54, 47, 83, 84, 9, 30, 11, 92, 63, 62, 75, 48, 23, 85, 23, 4, 31, 13, 98],
    [69, 30, 61, 35, 53, 98, 94, 33, 77, 31, 54, 71, 78, 9, 79, 51, 76, 56, 80, 72]
])



matrix = np.transpose(matrix)

neh_solution =neh_algorithm(matrix)

print(neh_solution[0])


[5, 6, 14, 11, 12, 10, 2, 15, 19, 3, 16, 0, 8, 1, 9, 7, 18, 4, 13, 17]


In [ ]:
initial_solution = neh_solution[0]
initial_cost = neh_solution[1]

start_time = time.time()
best_solution, best_cost, i = branch_and_bound(matrix, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'Results of Branch & Bound:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
#print(f'No. Nodes visited is {i}.')
print(f'Elapsed time of {elapsed_time} seconds.')

new best solution [5, 13, 16, 18, 9, 6, 8, 2, 3, 7, 17, 14, 10, 11, 12, 15, 19, 0, 4, 1]
new best cost 1365.0
new best solution [5, 18, 13, 9, 6, 2, 15, 19, 10, 16, 4, 17, 14, 11, 0, 7, 8, 12, 3, 1]
new best cost 1364.0
new best solution [5, 18, 9, 6, 2, 13, 15, 11, 10, 16, 17, 19, 4, 0, 12, 8, 14, 3, 7, 1]
new best cost 1360.0
new best solution [5, 9, 16, 6, 2, 13, 15, 3, 18, 14, 8, 7, 17, 19, 10, 0, 12, 11, 4, 1]
new best cost 1359.0
Results of Branch & Bound:
Best sequence is [5, 9, 16, 6, 2, 13, 15, 3, 18, 14, 8, 7, 17, 19, 10, 0, 12, 11, 4, 1] with a makespan of 1359.0.
Elapsed time of 0.16547703742980957 seconds.


# test instance 7

In [ ]:
import numpy as np

#-----------------------------------------------instance 7
matrix = np.array([
    [15, 64, 64, 48, 9, 91, 27, 34, 42, 3, 11, 54, 27, 30, 9, 15, 88, 55, 50, 57],
    [28, 4, 43, 93, 1, 81, 77, 69, 52, 28, 28, 77, 42, 53, 46, 49, 15, 43, 65, 41],
    [77, 36, 57, 15, 81, 82, 98, 97, 12, 35, 84, 70, 27, 37, 59, 42, 57, 16, 11, 34],
    [1, 59, 95, 49, 90, 78, 3, 69, 99, 41, 73, 28, 99, 13, 59, 47, 8, 92, 87, 62],
    [45, 73, 59, 63, 54, 98, 39, 75, 33, 8, 86, 41, 41, 22, 43, 34, 80, 16, 37, 94]
])



matrix = np.transpose(matrix)

neh_solution =neh_algorithm(matrix)

print(neh_solution[0])


[4, 2, 19, 10, 7, 5, 3, 8, 1, 12, 6, 18, 16, 9, 14, 15, 0, 17, 13, 11]


In [ ]:
initial_solution = neh_solution[0]
initial_cost = neh_solution[1]

start_time = time.time()
best_solution, best_cost, i = branch_and_bound(matrix, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'Results of Branch & Bound:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'Elapsed time of {elapsed_time} seconds.')

new best solution [9, 1, 12, 0, 18, 13, 15, 16, 8, 11, 19, 2, 7, 3, 10, 5, 4, 14, 6, 17]
new best cost 1279.0
new best solution [9, 1, 12, 0, 18, 13, 15, 16, 8, 19, 4, 3, 14, 11, 10, 5, 2, 7, 6, 17]
new best cost 1276.0
new best solution [9, 1, 12, 0, 18, 13, 15, 16, 8, 19, 4, 3, 10, 5, 2, 6, 14, 7, 11, 17]
new best cost 1272.0
new best solution [9, 1, 12, 0, 18, 13, 15, 16, 8, 19, 4, 6, 3, 2, 7, 5, 10, 14, 11, 17]
new best cost 1271.0
new best solution [9, 1, 12, 0, 18, 13, 15, 19, 10, 2, 3, 14, 16, 4, 5, 7, 8, 11, 6, 17]
new best cost 1265.0
new best solution [9, 1, 12, 0, 18, 13, 15, 19, 10, 2, 3, 16, 4, 5, 6, 8, 14, 7, 11, 17]
new best cost 1263.0
new best solution [9, 1, 12, 0, 18, 13, 19, 14, 3, 16, 8, 4, 7, 5, 10, 2, 6, 17, 11, 15]
new best cost 1262.0
new best solution [9, 1, 12, 0, 18, 13, 19, 14, 3, 16, 8, 4, 10, 5, 2, 7, 11, 17, 6, 15]
new best cost 1259.0
new best solution [9, 1, 12, 0, 18, 13, 19, 14, 15, 16, 8, 10, 3, 4, 11, 7, 5, 2, 6, 17]
new best cost 1258.0
new best s